# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Acuña  ->  L. Acuña  |  ['L. Acuña']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 55 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.16164


extracting tarball to tmp_2504.16164... done.


Found 194 bibliographic references in tmp_2504.16164/NOMADS.bbl.
Error retrieving bib data for mazeh2016dearth: 'author'
Error retrieving bib data for mazeh2016dearth: 'author'
Error retrieving bib data for castro2024mapping: 'author'
Error retrieving bib data for castro2024mapping: 'author'
Retrieving document from  https://arxiv.org/e-print/2504.16348


extracting tarball to tmp_2504.16348... done.
Retrieving document from  https://arxiv.org/e-print/2504.16648


/tmp/ipykernel_3246/2822249172.py:52: LatexWarning: 2504.16648 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2504.16881


extracting tarball to tmp_2504.16881...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.16164-b31b1b.svg)](https://arxiv.org/abs/2504.16164) | **Exploring the Neptunian Desert: Insights from a Homogeneous Planetary Sample**  |
|| L. Doyle, et al. -- incl., <mark>L. Acuña</mark> |
|*Appeared on*| *2025-04-24*|
|*Comments*| *Accepted and published in MNRAS. 18 pages, 9 figures, 5 tables*|
|**Abstract**|            In this paper, we present a homogeneous analysis of close-in Neptune planets. To do this, we compile a sample of TESS-observed planets using a ranking criterion which takes into account the planet's period, radius, and the visual magnitude of its host star. We use archival and new HARPS data to ensure every target in this sample has precise radial velocities. This yields a total of 64 targets, 46 of which are confirmed planets and 18 of which show no significant radial velocity signal. We explore the mass-radius distribution, planetary density, stellar host metallicity, and stellar and planetary companions of our targets. We find 26$\%$ of our sample are in multi-planet systems, which are typically seen for planets located near the lower edge of the Neptunian desert. We define a 'gold' subset of our sample consisting of 33 confirmed planets with planetary radii between 2$R_{\oplus}$ and 10$R_{\oplus}$. With these targets, we calculate envelope mass fractions (EMF) using the GAS gianT modeL for Interiors (GASTLI). We find a clear split in EMF between planets with equilibrium temperatures below and above 1300~K, equivalent to an orbital period of $\sim$3.5~days. Below this period, EMFs are consistent with zero, while above they typically range from 20$\%$ to 40$\%$, scaling linearly with the planetary mass. The orbital period separating these two populations coincides with the transition between the Neptunian desert and the recently identified Neptunian ridge, further suggesting that different formation and/or evolution mechanisms are at play for Neptune planets across different close-in orbital regions.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.16348-b31b1b.svg)](https://arxiv.org/abs/2504.16348) | **Supermassive Binaries in Ultralight Dark Matter Solitons**  |
|| R. Boey, E. Kendall, <mark>Y. Wang</mark>, R. Easther |
|*Appeared on*| *2025-04-24*|
|*Comments*| *14 pages, 20 figures*|
|**Abstract**|            Ultralight (or fuzzy) dark matter (ULDM) is an alternative to cold dark matter. A key feature of ULDM is the presence of solitonic cores at the centers of collapsed halos. These would potentially increase the drag experienced by supermassive black hole (SMBH) binaries, changing their merger dynamics and the resulting gravitational wave background. We perform detailed simulations of high-mass SMBH binaries in the soliton of a massive halo. We find more rapid decay than previous simulations and semi-analytic approximations. We confirm expectations that the drag depends strongly on the ULDM particle mass, finding masses greater than $10^{-21}$ eV could potentially alleviate the final parsec problem and that ULDM may even suppress gravitational wave production at lower frequencies in the pulsar timing band.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.16881-b31b1b.svg)](https://arxiv.org/abs/2504.16881) | **Fermi-LAT and FAST observation of the gamma-ray binary HESS J0632+057**  |
|| Y. Yang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-04-24*|
|*Comments*| **|
|**Abstract**|            Using 15 years of data from the Fermi Large Area Telescope (Fermi-LAT), we performed a comprehensive analysis on the gamma-ray binary HESS J0632+057. Its spectrum in 0.1-300 GeV band is well described by a power law model with an index of $2.40\pm0.16$, leading to an energy flux of (5.5$\pm$1.6$)\times$ 10$^{-12}$ erg cm$^{-2}$ s$^{-1}$. The GeV Spectral Energy Distribution (SED) of HESS J0632+057 hints for a spectral turn-over between $\sim$10-100 GeV. Orbital analysis reveals a flux enhancement during the phase range of 0.2-0.4, consistent with the X-ray and TeV light curves, indicating an origin of a common particle population. We carried out six deep radio observations on HESS J0632+057 with the Five-hundred-meter Aperture Spherical Telescope (FAST), evenly distributed across its orbit, reaching a detection sensitivity of 2$\mu$Jy. However, no radio pulsation was detected within these observations. The absence of radio pulsation may be attributed to the dense stellar wind environment of HESS J0632+057.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.16648-b31b1b.svg)](https://arxiv.org/abs/2504.16648) | **FAST Observation and Results for Core Collapse Globular Cluster M15 and NGC 6517**  |
|| Y. Wu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-24*|
|*Comments*| *4 pages, 3 figures, previously published work in URSI GASS*|
|**Abstract**|            Radio astronomy is part of radio science that developed rapidly in recent decades. In the research of radio astronomy, pulsars have always been an enduring popular research target. To find and observe more pulsars, large radio telescopes have been built all over the world. In this paper, we present our studies on pulsars in M15 and NGC 6517 with FAST, including monitoring pulsars in M15 and new pulsar discoveries in NGC 6517. All the previously known pulsars in M15 were detected without no new discoveries. Among them, M15C was still detectable by FAST, while it is assumed to fade out due to precession [1]. In NGC 6517, new pulsars were continues to be discovered and all of them are tend to be isolated pulsars. Currently, the number of pulsars in NGC 6517 is 17, much more than the predicted before [2].         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.16164/./period_radius_all_new.png', 'tmp_2504.16164/./period_radius_multilicity2.png', 'tmp_2504.16164/./EMF_vs_Period.png', 'tmp_2504.16164/./EMF_vs_planetarymass.png']
copying  tmp_2504.16164/./period_radius_all_new.png to _build/html/
copying  tmp_2504.16164/./period_radius_multilicity2.png to _build/html/
copying  tmp_2504.16164/./EMF_vs_Period.png to _build/html/
copying  tmp_2504.16164/./EMF_vs_planetarymass.png to _build/html/
exported in  _build/html/2504.16164.md
    + _build/html/tmp_2504.16164/./period_radius_all_new.png
    + _build/html/tmp_2504.16164/./period_radius_multilicity2.png
    + _build/html/tmp_2504.16164/./EMF_vs_Period.png
    + _build/html/tmp_2504.16164/./EMF_vs_planetarymass.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\Rearth}{R_{\oplus}}$
$\newcommand{\solar}{{\odot}}$
$\newcommand{\kep}{\it Kepler}$
$\newcommand{\tess}{\it TESS}$
$\newcommand{\ktwo}{\sl K2}$
$\newcommand{\swift}{\it Swift}$
$\newcommand{\Porb}{P_{\rm orb}}$
$\newcommand{\nuorb}{\nu_{\rm orb}}$
$\newcommand{\eplus}{\epsilon_+}$
$\newcommand{\eminus}{\epsilon_-}$
$\newcommand{\cd}{{\rm c d^{-1}}}$
$\newcommand{\MdotL}{\dot M_{\rm L1}}$
$\newcommand{\Mdot}{\dot M}$
$\newcommand{\Mdotsolar}{\dot{M_{\odot}} yr^{-1}}$
$\newcommand{\Ldisk}{L_{\rm disk}}$
$\newcommand{\src}{KIC 9202990}$
$\newcommand{\ergscm}{erg s^{-1} cm^{-2}}$
$\newcommand{\rchi}{\chi^{2}_{\nu}}$
$\newcommand{\chisq}{\chi^{2}}$
$\newcommand{\pcmsq}{cm^{-2}}$
$\newcommand$
$\newcommand{\rpl}{\mbox{R\textsubscript{p}}}$
$\newcommand{\rstar}{\mbox{R\textsubscript{*}}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Exploring the Neptunian Desert: Insights from a Homogeneous Planetary Sample

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.16164-b31b1b.svg)](https://arxiv.org/abs/2504.16164)<mark>Appeared on: 2025-04-24</mark> -  _Accepted and published in MNRAS. 18 pages, 9 figures, 5 tables_

</div>
<div id="authors">

L. Doyle, et al. -- incl., <mark>L. Acuña</mark>

</div>
<div id="abstract">

**Abstract:** In this paper, we present a homogeneous analysis of close-in Neptune planets. To do this, we compile a sample of TESS-observed planets using a ranking criterion which takes into account the planet's period, radius, and the visual magnitude of its host star. We use archival and new HARPS data to ensure every target in this sample has precise radial velocities. This yields a total of 64 targets, 46 of which are confirmed planets and 18 of which show no significant radial velocity signal. We explore the mass-radius distribution, planetary density, stellar host metallicity, and stellar and planetary companions of our targets. We find 26 \% of our sample are in multi-planet systems, which are typically seen for planets located near the lower edge of the Neptunian desert. We define a `gold' subset of our sample consisting of 33 confirmed planets with planetary radii between 2 $R_{\oplus}$ and 10 $R_{\oplus}$ . With these targets, we calculate envelope mass fractions (EMF) using the GAS gianT modeL for Interiors (GASTLI). We find a clear split in EMF between planets with equilibrium temperatures below and above 1300 K, equivalent to an orbital period of $\sim$ 3.5 days. Below this period, EMFs are consistent with zero, while above they typically range from 20 $\%$ to 40 $\%$ , scaling linearly with the planetary mass. The orbital period separating these two populations coincides with the transition between the Neptunian desert and the recently identified Neptunian ridge, further suggesting that different formation and/or evolution mechanisms are at play for Neptune planets across different close-in orbital regions.

</div>

<div id="div_fig1">

<img src="tmp_2504.16164/./period_radius_all_new.png" alt="Fig1" width="100%"/>

**Figure 1. -** Our full sample of all 64 targets within the period-radius space (black stars). The Neptunian desert boundaries are from mazeh2016dearth plotted as solid lines, with the enclosed desert area shaded in yellow. The dashed black lines represent the boundaries at 2 $\Rearth$ and 10 $\Rearth$ used to define the Neptune `gold' sample (black stars highlighted in gold). Known planets were sourced from the NASA exoplanet archive (\url{exoplanetarchive.ipac.caltech.edu}) accessed on 28 January 2025 and are plotted as grey dots in the background. The population density of known planets is shaded in green, where darker green denotes more planets discovered in that region of parameter space.  (*fig:nep_desert_all*)

</div>
<div id="div_fig2">

<img src="tmp_2504.16164/./period_radius_multilicity2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Our full sample of all 64 targets within the period-radius space. Targets identified as hosting a stellar companion (i.e. Stellar Companion(s)) are plotted with a light blue circle. Those which are members of a multiple planet system (i.e. Planetary Companion(s)) are plotted with a red highlight. Some targets meet both criteria. The Neptunian desert boundaries from mazeh2016dearth are plotted as solid lines, with the enclosed Neptunian desert area shaded in yellow. The updated Neptunian desert boundaries from castro2024mapping are plotted as a golden dashed line. All known planets were sourced from the NASA exoplanet archive (\url{https://www.exoplanetarchive.ipac.caltech.edu}) accessed on 28 January 2025 and are plotted as grey dots in the background. The population density of known planets is shaded in green, where darker green denotes more planets discovered in that region of parameter space.  (*fig:companions*)

</div>
<div id="div_fig3">

<img src="tmp_2504.16164/./EMF_vs_Period.png" alt="Fig6.1" width="50%"/><img src="tmp_2504.16164/./EMF_vs_planetarymass.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** _ Top:_ We show the distribution of Envelope Mass Fractions (EMF) against planetary orbital period for 29 of our `gold' sample of Neptunian desert planets. For reference, Saturn's EMF = 0.75-0.83 at M = 95.16 $M_{\oplus}$, while Neptune's EMF = 0.15-0.35 at M = 17.15 $M_{\oplus}$, depending on whether their measured atmospheric metallicities are used or a solar envelope is assumed \citep[see][for a detailed comparison with Solar System planets]{acuna2024gastli}. A dashed black line is shown at 3.5 and 5.7 days which represents the Neptune ridge defined in castro2024mapping. _ Bottom:_ The distribution of EMF against planetary mass for 29 of our `gold' sample of Neptunian desert planets. For each temperature range a linear trend in EMF has been fitted and shown as a dashed line. In both, the sample has been split according to the planetary equilibrium temperature with T$_{\rm{eq}}$ > 1300 K in yellow and T$_{\rm{eq}} \leq$ 1300 K in blue. Triangle markers represent planets where only an upper limit on the EMF was derived. (*fig:EMF*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.16164"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

450  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
